<a href="https://colab.research.google.com/github/LucasSerranog/IA_udea/blob/main/04_modelo_con_Random_Forest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Modelo con preprocesado alternativo y Random Forest

#integrante
Lucas Serrano CC.1000944832


En este notebook:
- Aplicaremos un preprocesado distinto al utilizado en el notebook 03.
- Entrenaremos un modelo Random Forest para predecir `RENDIMIENTO_GLOBAL`.
- Evaluaremos su rendimiento y lo compararemos con el modelo SVM.

Cargamos las librerias necesarias

In [1]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

subimos los datos preprocesados

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
ruta = "/content/drive/MyDrive/datos_preprocesados.csv"
d_p = pd.read_csv(ruta)


Se cargan los datos preprocesados

In [4]:
d_p

,FAMI_EDUCACIONMADRE__Educación profesional completa,FAMI_EDUCACIONMADRE__Educación profesional incompleta,FAMI_EDUCACIONMADRE__Ninguno,FAMI_EDUCACIONMADRE__Postgrado,FAMI_EDUCACIONMADRE__Primaria completa,FAMI_EDUCACIONMADRE__Primaria incompleta,FAMI_EDUCACIONMADRE__Secundaria (Bachillerato) completa,FAMI_EDUCACIONMADRE__Secundaria (Bachillerato) incompleta,FAMI_EDUCACIONMADRE__Técnica o tecnológica completa,FAMI_EDUCACIONMADRE__Técnica o tecnológica incompleta,...,ESTU_VALORMATRICULAUNIVERSIDAD__No pagó matrícula,FAMI_TIENEINTERNET__No,FAMI_TIENEINTERNET__Si,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,RENDIMIENTO_GLOBAL,coef_1,coef_2,coef_3,coef_4
0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,ENFERMERIA,BOGOTÁ,medio-alto,0.322,0.208,0.310,0.267
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,DERECHO,ATLANTICO,bajo,0.311,0.215,0.292,0.264
2,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,MERCADEO Y PUBLICIDAD,BOGOTÁ,bajo,0.297,0.214,0.305,0.264
3,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,ADMINISTRACION DE EMPRESAS,SANTANDER,alto,0.485,0.172,0.252,0.190
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,PSICOLOGIA,ANTIOQUIA,medio-bajo,0.316,0.232,0.285,0.294
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
692495,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,BIOLOGIA,LA GUAJIRA,medio-alto,0.237,0.271,0.271,0.311
692496,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,PSICOLOGIA,NORTE SANTANDER,bajo,0.314,0.240,0.278,0.260
692497,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,ADMINISTRACIÓN EN SALUD OCUPACIONAL,BOGOTÁ,medio-bajo,0.286,0.240,0.314,0.287
692498,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,PSICOLOGIA,TOLIMA,bajo,0.132,0.426,0.261,0.328


Eliminamos columnas irrelevantes como lo son el programa academico debido a la variabilidad de esos datos, al igual que el departamento

In [5]:
columnas_a_eliminar = ['ESTU_PRGM_ACADEMICO', 'ESTU_PRGM_DEPARTAMENTO']
d_p = d_p.drop(columns=columnas_a_eliminar)

### Separación de variables predictoras y variable objetivo

- **X**: contiene todas las columnas del DataFrame `d_p` excepto `'RENDIMIENTO_GLOBAL'`.  
  Estas columnas son las variables independientes que usaremos para predecir.

- **y**: contiene únicamente la columna `'RENDIMIENTO_GLOBAL'`, que es la variable dependiente.  
  Esta será la etiqueta o valor que el modelo debe aprender a predecir.


In [6]:
X = d_p.drop(columns='RENDIMIENTO_GLOBAL')
y = d_p['RENDIMIENTO_GLOBAL']

### División de datos

Se divide el conjunto de datos en dos partes: entrenamiento y prueba.

- `X_train`, `y_train`: se usarán para entrenar el modelo.
- `X_test`, `y_test`: se usarán para evaluar el modelo una vez entrenado.

**Parámetros usados:**
- `test_size=0.2`: el 20% de los datos se reserva para prueba.
- `random_state=42`: fija la semilla para obtener la misma división cada vez que se corre el código.
- `stratify=y`: asegura que la proporción de clases en la variable objetivo (`y`) se mantenga igual en ambos conjuntos (útil en problemas de clasificación).

In [7]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

### Definición del modelo Random Forest

Se crea una instancia del clasificador **RandomForestClassifier**, que es un algoritmo de aprendizaje automático basado en múltiples árboles de decisión (modelo de ensamble).

**Parámetros clave:**
- `random_state=42`: asegura que los resultados sean reproducibles.
- `n_jobs=-1`: permite usar todos los núcleos del procesador para paralelizar el entrenamiento, lo cual mejora el rendimiento.

In [8]:
modelo_rf = RandomForestClassifier(random_state=42, n_jobs=-1)


### Espacio de hiperparámetros amplio

Se define un diccionario llamado `param_dist` que contiene los diferentes valores que se desean explorar para ajustar el modelo **Random Forest**. Este conjunto de combinaciones será utilizado por un algoritmo como `RandomizedSearchCV` o `GridSearchCV` para encontrar los mejores hiperparámetros.

In [9]:
param_dist = {
    'n_estimators': [100, 200, 300, 400],
    'max_depth': [10, 20, 30, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False],
    'max_features': ['sqrt', 'log2']
}

### Búsqueda aleatoria de hiperparámetros con RandomizedSearchCV

Se utiliza `RandomizedSearchCV` para explorar múltiples combinaciones de hiperparámetros del modelo **Random Forest** y encontrar aquella que produzca el mejor desempeño (precisión).

In [10]:
random_search = RandomizedSearchCV(
    estimator=modelo_rf,
    param_distributions=param_dist,
    n_iter=10,
    cv=3,
    scoring='accuracy',
    verbose=2,
    random_state=42,
    n_jobs=-1
)


### Entrenamiento del modelo con búsqueda aleatoria

Se entrena el modelo `RandomForestClassifier` utilizando `RandomizedSearchCV`.  
Durante este proceso, se prueban 20 combinaciones aleatorias de hiperparámetros (`n_iter=10`) usando validación cruzada con 3 particiones (`cv=3`) para encontrar la configuración que brinde la mayor precisión.


In [11]:
random_search.fit(X_train, y_train)


Fitting 3 folds for each of 10 candidates, totalling 30 fits


/usr/local/lib/python3.11/dist-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


TerminatedWorkerError: A worker process managed by the executor was unexpectedly terminated. This could be caused by a segmentation fault while calling the function or by an excessive memory usage causing the Operating System to kill the worker.

The exit codes of the workers are {SIGKILL(-9)}
Detailed tracebacks of the workers should have been printed to stderr in the executor process if faulthandler was not disabled.